<h1>Machine learning using Neural networks to predict eye state data</h1>

<!-- <p>The purpose of this project is to get more familiar with an eeg dataset and learn more about data manipulation. The dataset is slipt into 15 differents columns with the last being the target values (0,1). The target values represent the state of the eye, being open or closed. The other data represents the value of an eeg node which gets the value of the electrical activity in that sector surrounding the node.<br> -->
This dataset is located in the UCI database found <a href='https://archive.ics.uci.edu/ml/datasets/EEG+Eye+State'>here</a>
</p>

<!-- <h2>The purpose of the nodes and locations</h2>
<p>The nodes used in this dataset are located in the red filled nodes shown below in Figure 1.<br>

</p> -->

<!-- <img src='data/eeg_eye_state_map.png' alt='eeg_eye_state_map'>Figure 1</img> -->


In [35]:
# import libraries
import numpy as np
import pandas as pd
from scipy.io.arff import loadarff
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from matplotlib import pyplot
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import KFold

In [5]:
# Load in the data frame
raw_data = loadarff('data/EEG Eye State.arff')
df_data = pd.DataFrame(raw_data[0])
df_data.head()

,AF3,F7,F3,FC5,T7,P7,O1,O2,P8,T8,FC6,F4,F8,AF4,eyeDetection
0,4329.23,4009.23,4289.23,4148.21,4350.26,4586.15,4096.92,4641.03,4222.05,4238.46,4211.28,4280.51,4635.90,4393.85,b'0'
1,4324.62,4004.62,4293.85,4148.72,4342.05,4586.67,4097.44,4638.97,4210.77,4226.67,4207.69,4279.49,4632.82,4384.10,b'0'
2,4327.69,4006.67,4295.38,4156.41,4336.92,4583.59,4096.92,4630.26,4207.69,4222.05,4206.67,4282.05,4628.72,4389.23,b'0'
3,4328.72,4011.79,4296.41,4155.90,4343.59,4582.56,4097.44,4630.77,4217.44,4235.38,4210.77,4287.69,4632.31,4396.41,b'0'
4,4326.15,4011.79,4292.31,4151.28,4347.69,4586.67,4095.90,4627.69,4210.77,4244.10,4212.82,4288.21,4632.82,4398.46,b'0'


<!-- <p>In order to pre-process the data I split the data into two sets. The data and the targets. The targets here are 1 or 0, meaning is the eye open or closed?</p> -->

In [21]:
# Preprocess Data
data_targets=df_data.iloc[:,-1]
data=df_data.iloc[:,:-1]


# convert targets into 1 or 0
s = pd.Series(data_targets)
data_targets=pd.get_dummies(s)
data_targets=data_targets.iloc[:,1]
print(data_targets)

0        0
1        0
2        0
3        0
4        0
        ..
14975    1
14976    1
14977    1
14978    1
14979    1
Name: b'1', Length: 14980, dtype: uint8


<!-- <p>In order to slipt into training and test data I use the method train_test_split() which randomizes the data while splitting it into a training and testing data set.</p> -->

In [27]:
# split the data
xTrain, xTest, yTrain, yTest = train_test_split(data, data_targets, test_size = 0.2, random_state = 25)

In [36]:
# standardize data 
scaler = StandardScaler()
scaler.fit(xTrain)
xTrain=scaler.transform(xTrain)
xTest=scaler.transform(xTest)

In [37]:
# evaluate model
clf=MLPClassifier(max_iter=2500, hidden_layer_sizes=(100, 5), random_state=1, solver='adam')
# clf=KNeighborsClassifier(n_neighbors=3)
clf.fit(xTrain, yTrain)
predictions =clf.predict(xTest)
print(predictions)
print(accuracy_score(yTest, predictions))

[1 0 0 ... 0 0 1]
0.8724966622162884
